In [9]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import janitor #pyjanitor 
%matplotlib inline
pd.set_option("display.max_columns", 101)

In [42]:
df = pd.read_csv('assignment5_q2_cleaned.csv').clean_names()
df

,unnamed_0,tcad_or_wcad_property_id_s_,duct_system_1_type,duct_system_1_rvalue,duct_system_1_return_sizing,duct_system_1_leakage,system_1_location_air_handler,system_1_air_handler_type,system_1_age_years_,system_1_sqft_ton,water_heater_fuel_type,water_heating_tank_type,toilet_type,programmable_thermostat_present,window_screen_area_recommended_sqft_,furnace_fuel_type,attic_r_value,recommended_additional_r_value,year_built,bedrooms,conditioned_sqft_,attic_sqft_,home_type,lat,long,set,needsductfixed
0,5,574898,Mylar_Flex,4.0,Adequate,7.0,Closet,Up Flow,15,542.0,Elec,Tank,Water_Saving,Y,55.0,Gas,25.0,0.0,1982,3.0,1356,1036,Single Family,30.192786,-97.826929,train,0
1,6,534406,Mylar_Flex,6.0,Adequate,19.0,Attic,Horizontal,11,540.0,Gas,Tank,Water_Saving,Y,0.0,Gas,30.0,8.0,2002,4.0,2160,2160,Single Family,30.145708,-97.811652,train,0
2,7,216111,Mylar_Flex,8.0,Adequate,14.0,Attic,Horizontal,17,435.0,Elec,Tank,Water_Saving,Y,116.0,Elec,19.0,19.0,1934,2.0,870,870,Single Family,30.307635,-97.721333,train,0
3,10,207026,Mylar_Flex,6.0,Adequate,21.0,Attic,Horizontal,17,524.0,Gas,Tank,Water_Saving,N,0.0,Gas,19.0,19.0,1939,2.0,1064,1064,Single Family,30.286246,-97.722137,test,0
4,11,204835,Sheet_Metal,4.0,Adequate,58.0,Closet,Up Flow,11,496.0,Elec,Tank,Water_Saving,N,0.0,Gas,22.0,16.0,1968,3.0,1242,0,Single Family,30.347212,-97.761808,train,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276,17303,259537,Mylar_Flex,8.0,Inadequate,14.3,Garage,Up Flow,13,518.0,Elec,Tank,Water_Saving,Y,0.0,Elec,56.0,0.0,1986,3.0,1814,1814,Single Family,30.386068,-97.665090,train,0
11277,17304,362455,Mylar_Flex,8.0,Adequate,18.0,Attic,Horizontal,10,556.0,Gas,Tank,Water_Saving,Y,0.0,Gas,26.4,11.6,1994,5.0,3063,1531,Single Family,30.413203,-97.683318,train,0
11278,17305,497547,Sheet_Metal,5.0,Adequate,24.0,Closet,Up Flow,11,550.0,Gas,Tank,Water_Saving,Y,0.0,Gas,22.0,16.0,1968,3.0,2207,1850,Single Family,30.382395,-97.749469,test,0
11279,17310,219191,Sheet_Metal,6.0,Adequate,18.0,Closet,Up Flow,6,442.0,Gas,Tank,Water_Saving,Y,140.0,Gas,26.0,0.0,1962,5.0,3418,1143,Single Family,30.307164,-97.675189,test,0


In [49]:
x = df[['duct_system_1_type', 'system_1_location_air_handler', 'system_1_air_handler_type','system_1_age_years_', 'furnace_fuel_type', 'home_type' ]]


x = pd.concat([x,pd.get_dummies(x['duct_system_1_type'], prefix='duct_type'), pd.get_dummies(x['system_1_location_air_handler'], prefix='location'),
              pd.get_dummies(x['system_1_air_handler_type'], prefix='handler_type'), pd.get_dummies(x['furnace_fuel_type'], prefix='furnace'),
              pd.get_dummies(x['home_type'], prefix='')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
x.drop(['duct_system_1_type', 'system_1_location_air_handler', 'system_1_air_handler_type', 'furnace_fuel_type', 'home_type'],axis=1, inplace=True)
x.describe()

,system_1_age_years_,duct_type_Duct_Board,duct_type_Grey_Flex,duct_type_Mylar_Flex,duct_type_Sheet_Metal,location_Attic,location_Closet,location_Garage,handler_type_Down Flow,handler_type_Horizontal,handler_type_Up Flow,furnace_Elec,furnace_Gas,_Condo,_Duplex,_Fourplex,_Single Family,_Town Homes,_Triplex
count,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000,11281.000000
mean,13.019413,0.182874,0.174009,0.443489,0.199628,0.373903,0.496321,0.099814,0.003812,0.372573,0.604645,0.411932,0.588068,0.049996,0.056821,0.011524,0.864019,0.017197,0.000443
std,5.683174,0.386580,0.379134,0.496818,0.399738,0.483860,0.500009,0.299765,0.061624,0.483511,0.488948,0.492205,0.492205,0.217945,0.231511,0.106733,0.342783,0.130011,0.021049
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,18.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,30.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
y = df[['duct_system_1_leakage']]
y

,duct_system_1_leakage
0,7.0
1,19.0
2,14.0
3,21.0
4,58.0
...,...
11276,14.3
11277,18.0
11278,24.0
11279,18.0


In [51]:
x = x.values.reshape(11281, 19)
y = y.values.reshape(11281, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.50, random_state=42)

In [52]:
DecisionTreeRegModel = DecisionTreeRegressor()
DecisionTreeRegModel.fit(x_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [53]:
y_pred = DecisionTreeRegModel.predict(x_test)
y_pred

array([14.        , 14.684375  , 18.01666667, ..., 13.13478261,
       16.46666667, 39.26666667])

In [55]:
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse) # RMSD serves to aggregate the magnitudes of the errors in predictions for various times into a single measure of predictive power.
rmse # good for comparing similar models, but if different data is used it is worthless because RMSE is dependent on the scale that was used 

11.791739379889554